In [3]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

In [7]:
#Read in Data from CSV
drug_data_csv = "Resources/drug_deaths.csv"
income_data_csv = "Resources/per_capita_personal_income.csv"
drug_data = pd.read_csv(drug_data_csv)
income_data = pd.read_csv(income_data_csv)
#drug_data.head()
income_data.head()

,GeoFips,GeoName,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,...,2016:Q3,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,0,United States,43971,44381,44191,45616,44353,44720,44910,45171,...,49734,50134,50794,51286,51773,52428,53022,53532,54122,54578
1,1000,Alabama,35571,35873,35507,36323,35900,36009,36176,35959,...,39004,39243,39823,40112,40305,40731,41148,41409,41675,42162
2,2000,Alaska *,53535,53819,53460,54042,52634,52545,53053,53253,...,56218,56553,56564,56869,57597,58164,58846,59239,60258,60717
3,4000,Arizona,36104,36470,36217,37364,36701,37090,37254,37427,...,41720,41857,42608,42966,43597,43955,44688,44994,45551,45890
4,5000,Arkansas,35988,36556,36187,37595,36500,36639,36803,36739,...,40915,41214,41757,41871,42027,42567,42986,43338,43849,44447


In [ ]:
#Desired Data
new_drug_data = drug_data[['ID','Date', 'Age', 'ResidenceState', 'COD', 'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl_Analogue','Oxycodone','Oxymorphone', 'Ethanol','Hydrocodone', 'Benzodiazepine', 'Methadone','Amphet', 'Tramad','Morphine_NotHeroin', 'Hydromorphone','Other']].copy()
#new_drug_data.head()

new_income_data = income_data.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ,12 ,13, 14, 15, 16,17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,28, 29]]
#new_income_data

In [ ]:
#Adjust Dates for both data sets to yearly

In [ ]:
#Store into a DF

In [ ]:
#Connect to Local DB

In [ ]:
#check for tables, join tables, query, etc.